# Real Estate Web Scraper

In [ ]:
# headers for get method for Idealista website
headers_idealista = {
                      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,'\
                          'image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                      'accept-encoding': 'gzip, deflate, br',
                      'accept-language': 'pt-PT,pt;q=0.9,en-US;q=0.8,en;q=0.7',
                      'cache-control': 'max-age=0',
                      'referer': 'https://www.idealista.pt/en/comprar-casas/lisboa/?ordem=atualizado-desc',
                      'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
                      'sec-ch-ua-mobile': '?0',
                      'sec-fetch-dest': 'document',
                      'sec-fetch-mode': 'navigate',
                      'sec-fetch-site': 'same-origin',
                      'sec-fetch-user': '?1',
                      'upgrade-insecure-requests': '1',
                      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '\
                          'Chrome/91.0.4472.77 Safari/537.36'
                  }

In [ ]:
from requests import get
from bs4 import BeautifulSoup

In [ ]:
# I wanted to get the information needed to build the dictionary
# from the downloaded html page, so that I'm not continuously making requests
# html_file = open("/content/drive/MyDrive/Colab Notebooks/idealista.html", "r")
# html_soup = BeautifulSoup(html_file.read(), 'html.parser')
# base_url = "https://www.idealista.pt"
# parish_name = "Lumiar"
# things used while creating the method below

In [ ]:
# For guidance:
# t2_campolide = {'ad_title': "Apartamento T2 em Centro - Nova Campolide, Campolide",
#                'ad_id': 29929511,
#                'ad_link': "https://www.idealista.pt/imovel/29929511/)",
#                'house_typology': "T2",
#                'house_price_euros': 735000,
#                'house_area_m2: 106,
#                'house_floor': '1º andar',
#                'house_parish': "Campolide",
#                'seller_contact': 210020965}

def get_houses_per_subparish(url, parish_name, houses_dict):
  base_url = "https://www.idealista.pt" # should I give base_url as arg?
  response = get(url, headers=headers_idealista)
  html_soup = BeautifulSoup(response.text, 'html.parser')
  houses_soup = html_soup.find_all('article', class_="item")
  for house_soup in houses_soup:
    house_dict = {}
    house_dict['ad_id'] = int(house_soup['data-adid'])
    house_dict['ad_title'] = house_soup.find('a', class_='item-link')['title']
    house_dict['ad_link'] = base_url + house_soup.find('a', class_='item-link')['href']
    house_details = house_soup.find_all('span', class_='item-detail')
    house_dict['house_typology'] = house_details[0].text
    # is there a better way other than descentants?
    house_dict['house_area_m2'] = int(list(house_details[1].descendants)[0].replace('.', ''))
    try:
      house_dict['house_floor'] = list(house_details[2].descendants)[0]
    except:
      house_dict['house_floor'] = ""
    house_dict['house_price_euros'] = int(list(house_soup.find('span', class_='item-price').descendants)[0].replace('.', ''))
    house_dict['house_parish'] = parish_name
    try:
      house_dict['seller_contact'] = int(house_soup.find('span', class_='icon-phone item-not-clickable-phone').text)
    except:
      house_dict['seller_contact'] = ""
    houses_dict.append(house_dict)
  if html_soup.find('li', class_="next"):
    url = base_url + html_soup.find('li', class_="next").a['href']
    get_houses_per_subparish(url, parish_name, houses_dict)


In [ ]:
# there's a maximum of 30 houses per page
parishes = [
            {
                'name': 'Campolide',
                'slug': 'campolide',
                'sub_parishes': ['bairro-da-serafina', 'praca-de-espanha-sete-rios', 'bairro-calcada-dos-mestres', 'centro-nova-campolide']
            },
            {
                'name': 'Lumiar',
                'slug': 'lumiar',
                'sub_parishes': ['paco-do-lumiar', 'quinta-dos-alcoutins', 'carriche-hospital-das-forcas-armadas',
                            'alto-do-lumiar-quinta-das-conchas-quinta-do-lambert', 'centro', 'telheiras']
            },
            {
                'name': 'São Domingos de Benfica',
                'slug': 'sao-domingos-de-benfica',
                'sub_parishes': ['alto-dos-moinhos', 'laranjeiras', 'estrada-de-benfica', 'sete-rios']
            }
           ]

def get_houses_idealista(parishes):
  base_url = "https://www.idealista.pt"
  houses = []
  for parish in parishes:
    for sub_parish in parish['sub_parishes']:
      url = base_url + '/comprar-casas/' + parish['slug'] + '/' + sub_parish + '/'
      get_houses_per_subparish(url, parish['name'], houses)
  return houses

houses_idealista = get_houses_idealista(parishes)

In [ ]:
len(houses_idealista)

530

In [ ]:
houses_idealista[0]

{'ad_id': 31684620,
 'ad_link': 'https://www.idealista.pt/imovel/31684620/',
 'ad_title': 'Moradia em Bairro da Serafina, Campolide',
 'house_area_m2': 103,
 'house_floor': '',
 'house_parish': 'Campolide',
 'house_price_euros': 118000,
 'house_typology': 'T7 ',
 'seller_contact': 215561507}